In [ ]:
pip install msal

In [ ]:
import msal

In [ ]:
# Função de escrita no Azure SQL

def write_AzureSQLdb():
    
    # ID do locatário localizado nos Registros de Aplicativos do Azure Active Directory -> app-user-databricks
    tenant_id = 'b4920c82-7581-491a-9dab-cd2ade2f3ebd'
    
    #Autoridade
    msal_authority = 'https://login.windows.net/' + tenant_id
    
    # URL do recurso DB
    resource_url = 'https://database.windows.net/.default'
    
    #Service Principal Client ID -> Criado em secrets no Key Vault
    service_principal_id = dbutils.secrets.get(scope = 'scope-adb-estudos', key = 'app-reg-adb')
    
    #Service Principal Secret -> Criado em secrets no Key Vault
    service_principal_secret = dbuilts.secrets.get(scope = 'scope-adb-estudos', key = 'app-user-databricks')
    
    #Variáveis e string de conexão ao Servidor do Azure SQL
    
    jdbcHostname = 'sql-estudo.database.windows.net'
    jdbcDatabase = 'db-estudos'
    jdbcPort = 1433
    jdbcUrl = f'jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database = {jdbcDatabase}'
    
    # Criando a instância que irá ser utilizada durante o ciclo de vida do aplicativo
    
    instance = msal.ConfidentialClientApplication(service_principal_id, service_principal_secret, msal_authority)
    
    # Envio de solicitação ao AAD para obter um token
    
    token = instance.acquire_token_for_client(resource_url)
    
    # Acessando o token
    
    access_token = token['access_token']
    
    # Selecionando colunas, o formato da conexão, url da conexão,  schema.tabela e acesso do token
    
    df.select('CotacaoCompra',
             'DataHoraCotacao',
             ).write.format('jdbc')\
                        .mode('append')\
                        .option('url', jdbcUrl)\
                        .option('dbtable',
                                'dolar_marcus_araujo.dolar_procedure_marcus_araujo02')\
                        .option('accesstoken', access_token)\
                        .save()